# Object Detection with Bounding Box Regression

This notebook will demonstrate some of the basic steps in object detection. Bounding box regression, as the name suggests, makes a regressor model which directly outputs the bounding box coordinates given an image.

This concept has been internal to object detectors like [YOLO](https://arxiv.org/abs/1506.02640).

Note: If you are running this notebook on Colab itself, I insist you turn on the GPU runtime. Go to **Tools ~> Runtime ~> Change runtime type ~> Hardware Accelerator ~> Select "GPU"**

Let's Start!




## 1) Importing the packages

We import TensorFlow and Keras along with our beloved NumPy. We print the TensorFlow version we're using.


In [0]:

from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow as tf
import numpy as np

print( tf.VERSION )


1.14.0


## 2) Downloading the data from GitHub

We need to download our dataset from the [Dataset_Archives](https://github.com/shubham0204/Dataset_Archives) repo where I have hosted a ZIP file of the dataset. The original dataset hails from [Kaggle.com](https://www.kaggle.com) as [Image Localization Dataset](https://www.kaggle.com/mbkinaci/image-localization-dataset) by [Muhammed Buyukkinaci](https://www.kaggle.com/mbkinaci).

Also, we install the [xmltodict](https://pypi.org/project/xmltodict/) package from PyPI which will be useful to parse the XML files containing the bounding box annotations.


In [0]:

!pip install xmltodict

import requests, zipfile, io

r = requests.get( 'https://github.com/shubham0204/Dataset_Archives/blob/master/image-localization-dataset.zip?raw=true' ) 
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()


## 3) Defining useful variables

Below are some variables which help us in training and preprocessing steps.

1.   `input_dim` : The dimensions of input image.
2.   `num_classes` : The number of classes we need to predict.
3.   `pred_vector_length` : This will be the length of the final output of our CNN. The vector will consists of 4 bounding box coordinates and 3 class probabilities ( 4 + 3 = 7 ).



In [0]:

input_dim = 228
num_classes = 3
pred_vector_length = 4 + num_classes


## 4) Parsing the images from the dataset

We prepare a list of all the files under `training_images` folder which have a `*.jpg` extension ( image files ) using `glob`. We resize them to the `input_dim` and normalize them.

In [0]:

from PIL import Image , ImageDraw
import os
import glob

images = []
image_paths = glob.glob( 'training_images/*.jpg' )
for imagefile in image_paths:
    image = Image.open( imagefile ).resize( ( input_dim , input_dim ))
    image = np.asarray( image ) / 255.0
    images.append( image )


## 5) Parsing the bounding box annotations

We parse all XML files under the `training_images` folder. These annotations are in the popular `PASCAL-VOC` format. We extract the four bounding box coordinates and normalize them using `input_dim`. Also, we extract the class labels from those files.

In [0]:

import xmltodict
import os

bboxes = []
classes_raw = []
annotations_paths = glob.glob( 'training_images/*.xml' )
for xmlfile in annotations_paths:
    x = xmltodict.parse( open( xmlfile , 'rb' ) )
    bndbox = x[ 'annotation' ][ 'object' ][ 'bndbox' ]
    bndbox = np.array([ int(bndbox[ 'xmin' ]) , int(bndbox[ 'ymin' ]) , int(bndbox[ 'xmax' ]) , int(bndbox[ 'ymax' ]) ])
    bndbox2 = [ None ] * 4
    bndbox2[0] = bndbox[0]
    bndbox2[1] = bndbox[1]
    bndbox2[2] = bndbox[2]
    bndbox2[3] = bndbox[3]
    bndbox2 = np.array( bndbox2 ) / input_dim
    bboxes.append( bndbox2 )
    classes_raw.append( x[ 'annotation' ][ 'object' ][ 'name' ] )



## 6) Preparing the final datasets

Our input will be an array of all the images we parsed earlier. The target output will be a concatenation of two arrays i.e the bounding box coordinates and the onehot class labels.

Using scikit-learn's `train_test_split` method, we split our dataset in training and validation datasets.



In [0]:

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

boxes = np.array( bboxes ) 
encoder = LabelBinarizer()
classes_onehot = encoder.fit_transform( classes_raw )

Y = np.concatenate( [ boxes , classes_onehot ] , axis=1 )
X = np.array( images )

print( X.shape ) 
print( Y.shape )

x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.1 )


(186, 228, 228, 3)
(186, 7)


## 7) Defining the loss function and evaluation metrics

First, we define a method which calculates the intersection over union ( IOU ) of two bounding boxes. In the `custom_loss`, we first calculate Mean Squared Error of the target and predicted bounding boxes. Next, we calculate the IOU loss which is $1 - iou$.

Hence, our final loss function is like,

$\Large L( y , y' ) = MSE( y, y') + ( 1 - IOU( y , y' ))$

Also, we use IOU as a validation metric to evaluate the model's performance on the testing dataset.

**Note: The loss function is a custom implementation. IOU is mostly used as a metric and not in the loss function.**



In [0]:

input_shape = ( input_dim , input_dim , 3 )
dropout_rate = 0.5
alpha = 0.2

def calculate_iou( target_boxes , pred_boxes ):
    xA = K.maximum( target_boxes[ ... , 0], pred_boxes[ ... , 0] )
    yA = K.maximum( target_boxes[ ... , 1], pred_boxes[ ... , 1] )
    xB = K.minimum( target_boxes[ ... , 2], pred_boxes[ ... , 2] )
    yB = K.minimum( target_boxes[ ... , 3], pred_boxes[ ... , 3] )
    interArea = K.maximum( 0.0 , xB - xA ) * K.maximum( 0.0 , yB - yA )
    boxAArea = (target_boxes[ ... , 2] - target_boxes[ ... , 0]) * (target_boxes[ ... , 3] - target_boxes[ ... , 1])
    boxBArea = (pred_boxes[ ... , 2] - pred_boxes[ ... , 0]) * (pred_boxes[ ... , 3] - pred_boxes[ ... , 1])
    iou = interArea / ( boxAArea + boxBArea - interArea )
    return iou

def custom_loss( y_true , y_pred ):
    mse = tf.losses.mean_squared_error( y_true , y_pred ) 
    iou = calculate_iou( y_true , y_pred ) 
    return mse + ( 1 - iou )

def iou_metric( y_true , y_pred ):
    return calculate_iou( y_true , y_pred ) 
    

## 8) Making the CNN model

We'll use a CNN model which takes in input an image of shape `( input_dim , input_dim , 3 )` and outputs a vector of shape `( 7 , )`.

*   The `Conv2D` layers will extract features from the image.
*   We'll use `Dropout` and `LeakyReLU` activation layer to avoid overfitting ( the model will then predict the same bounding boxes for all images ).
*   The `Dense` layers will be trained to output the final bounding boxes.

We use a small learning rate of 0.0001 so that the learning doesn't stop. Also, we pass the `iou_metric` in the `model.compile()` method.



In [0]:

model_layers = [       
	keras.layers.Conv2D(16, kernel_size=(3, 3), strides=1, input_shape=input_shape),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(16, kernel_size=(3, 3), strides=1 ),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(128, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(256, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),

    keras.layers.Flatten() , 

    keras.layers.Dense( 1240 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 640 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 480 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 120 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 62 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,

    keras.layers.Dense( pred_vector_length ),
    keras.layers.LeakyReLU( alpha=alpha ) ,
]

model = keras.Sequential( model_layers )
model.compile(
	optimizer=keras.optimizers.Adam( lr=0.0001 ),
	loss=custom_loss,
    metrics=[ iou_metric ]
)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## 9) Training the model

We finally train the model with the validation dataset and save it to the disk.

In [0]:

model.fit( 
    x_train ,
    y_train , 
    validation_data=( x_test , y_test ),
    epochs=200 ,
    batch_size=3 
)

model.save_weights( 'pretrained_weights.h5')


Train on 167 samples, validate on 19 samples
Epoch 1/200
167/167 [==============================] - 2s 12ms/sample - loss: 0.1372 - iou_metric: 0.8670 - val_loss: 0.5550 - val_iou_metric: 0.5756
Epoch 2/200
167/167 [==============================] - 2s 11ms/sample - loss: 0.1182 - iou_metric: 0.8851 - val_loss: 0.5591 - val_iou_metric: 0.5667
Epoch 3/200
167/167 [==============================] - 2s 11ms/sample - loss: 0.1217 - iou_metric: 0.8824 - val_loss: 0.5526 - val_iou_metric: 0.5721
Epoch 4/200
167/167 [==============================] - 2s 11ms/sample - loss: 0.1200 - iou_metric: 0.8841 - val_loss: 0.5352 - val_iou_metric: 0.5848
Epoch 5/200
167/167 [==============================] - 2s 11ms/sample - loss: 0.1237 - iou_metric: 0.8807 - val_loss: 0.5346 - val_iou_metric: 0.5939
Epoch 6/200
167/167 [==============================] - 2s 11ms/sample - loss: 0.1187 - iou_metric: 0.8845 - val_loss: 0.5591 - val_iou_metric: 0.5681
Epoch 7/200
167/167 [==============================] - 

## 10) Testing the model on test images

The following code will predict bounding boxes for some unseen images, draw them on the image using `ImageDraw` and then save them to a folder ( created with `mkdir` command ).

In [0]:

!mkdir -v inference_images

boxes = model.predict( x_test )
for i in range( boxes.shape[0] ):
    b = boxes[ i , 0 : 4 ] * input_dim
    img = x_test[i] * 255
    source_img = Image.fromarray( img.astype( np.uint8 ) , 'RGB' )
    draw = ImageDraw.Draw( source_img )
    draw.rectangle( b , outline="black" )
    source_img.save( 'inference_images/image_{}.png'.format( i + 1 ) , 'png' )


mkdir: created directory 'inference_images'


## 11) Print the average IOU and class accuracy

We will calculate the average IOU score over the validation dataset and also the accuracy of the model in predicting classes.

In [0]:

def calculate_avg_iou( target_boxes , pred_boxes ):
    xA = np.maximum( target_boxes[ ... , 0], pred_boxes[ ... , 0] )
    yA = np.maximum( target_boxes[ ... , 1], pred_boxes[ ... , 1] )
    xB = np.minimum( target_boxes[ ... , 2], pred_boxes[ ... , 2] )
    yB = np.minimum( target_boxes[ ... , 3], pred_boxes[ ... , 3] )
    interArea = np.maximum(0.0, xB - xA ) * np.maximum(0.0, yB - yA )
    boxAArea = (target_boxes[ ... , 2] - target_boxes[ ... , 0]) * (target_boxes[ ... , 3] - target_boxes[ ... , 1])
    boxBArea = (pred_boxes[ ... , 2] - pred_boxes[ ... , 0]) * (pred_boxes[ ... , 3] - pred_boxes[ ... , 1])
    iou = interArea / ( boxAArea + boxBArea - interArea )
    return iou

def class_accuracy( target_classes , pred_classes ):
    target_classes = np.argmax( target_classes , axis=1 ) 
    pred_classes = np.argmax( pred_classes , axis=1 ) 
    return ( target_classes == pred_classes ).mean()

target_boxes = y_test * input_dim
pred = model.predict( x_test )
pred_boxes = pred[ ... , 0 : 4 ] * input_dim
pred_classes = pred[ ... , 4 : ]

iou_scores = calculate_avg_iou( target_boxes , pred_boxes )
print( 'Mean IOU score {}'.format( iou_scores.mean() ) )

print( 'Class Accuracy is {} %'.format( class_accuracy( y_test[ ... , 4 : ] , pred_classes ) * 100 ))


Mean IOU score 0.5800086476915025
Class Accuracy is 42.10526315789473 %
